Importing necessary libraries

In [2]:
import cv2
import os
import numpy as np
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten  , Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report,confusion_matrix

In [6]:
#Defining various classes
classes=['Drought','Earthquake','Fire_Disaster','Human_Damage','Land_Slide','Non_Damage','Water_Disaster']

DATASET_PATH = "Dataset"

#Spilting dataset
Train_Split=0.75
Val_Split=0.1
Test_Split=0.25

imagePaths = list(paths.list_images(DATASET_PATH))
data = []
labels = []

#Processing Images
for imagePath in imagePaths:
    label = imagePath.split(os.path.sep)[1]
    image = cv2.imread(imagePath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (200, 200))
    data.append(image)
    labels.append(label)

print(labels)

['Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land_Slide', 'Land

In [7]:
#Processing Data
data = np.array(data)
labels = np.array(labels)

# perform one-hot encoding on the labels
lb = LabelBinarizer()
labels = lb.fit_transform(labels)

# partition the data into training and testing splits
(trainX, testX, trainY, testY) = train_test_split(data, labels,
    test_size=Test_Split, random_state=42)
# take the validation split from the training split
(trainX, valX, trainY, valY) = train_test_split(trainX, trainY,
    test_size=Val_Split, random_state=84)

Creating Model

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(200,200,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=32,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=64, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))

model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(units=512,activation="relu"))

model.add(Dense(units=7, activation="softmax"))

#compiling model
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=categorical_crossentropy, metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 200, 200, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 200, 200, 32)      18464     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 25, 25, 128)       0

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Traning model

In [ ]:
checkpoint = ModelCheckpoint("vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq=1)
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit(x=trainX,y=trainY,epochs=10,batch_size=32,callbacks=[checkpoint,early],validation_data=(valX,valY))

Epoch 1/10
286/286 [==============================] - 2197s 8s/step - loss: 6.2468 - accuracy: 0.6871 - val_loss: 0.6950 - val_accuracy: 0.7974
Epoch 2/10
286/286 [==============================] - 2202s 8s/step - loss: 0.7271 - accuracy: 0.7694 - val_loss: 0.5606 - val_accuracy: 0.8250
Epoch 3/10
286/286 [==============================] - 2202s 8s/step - loss: 0.5163 - accuracy: 0.8221 - val_loss: 0.5200 - val_accuracy: 0.8466
Epoch 4/10
286/286 [==============================] - 2204s 8s/step - loss: 0.4045 - accuracy: 0.8605 - val_loss: 0.4608 - val_accuracy: 0.8614
Epoch 5/10
286/286 [==============================] - 2210s 8s/step - loss: 0.2981 - accuracy: 0.8969 - val_loss: 0.5059 - val_accuracy: 0.8486
Epoch 6/10
286/286 [==============================] - 2220s 8s/step - loss: 0.1965 - accuracy: 0.9308 - val_loss: 0.5796 - val_accuracy: 0.8456
Epoch 7/10
286/286 [==============================] - 2213s 8s/step - loss: 0.1649 - accuracy: 0.9503 - val_loss: 0.5851 - val_accuracy:

In [ ]:
# model.save("Natural_Disaster")

INFO:tensorflow:Assets written to: Natural_Disaster/assets


Load Model

In [5]:
model = keras.models.load_model('Natural_Disaster')

model.summary()

In [ ]:
#Evalution on testing data
print("Evaluate on test data")
results = model.evaluate(testX, testY, batch_size=128)
print("test loss, test acc:", results)

# Generating predictions on new data 
print("Generated predictions for 3 samples")
predictions = model.predict(testX[:3])
print("predictions shape:", predictions.shape)

Evaluate on test data
27/27 [==============================] - 134s 5s/step - loss: 0.2842 - accuracy: 0.9428
test loss, test acc: [0.2842477858066559, 0.9427559971809387]
Generated predictions for 3 samples
predictions shape: (3, 7)


In [ ]:
for i in predictions:
    i=i.argmax()
    print(classes[i])

Water_Disaster
Non_Damage
Earthquake


Classification Report

In [9]:
y_pred = model.predict(testX, batch_size=64, verbose=0)
y_pred_bool = np.argmax(y_pred, axis=1)
testY = np.argmax(testY, axis=1)
print(classification_report(testY, y_pred_bool))

              precision    recall  f1-score   support

           0       0.89      0.74      0.80        53
           1       0.85      0.84      0.84       354
           2       0.89      0.88      0.89       232
           3       0.97      0.54      0.69        65
           4       0.88      0.72      0.79       123
           5       0.99      0.99      0.99      2308
           6       0.71      0.89      0.79       254

    accuracy                           0.94      3389
   macro avg       0.88      0.80      0.83      3389
weighted avg       0.94      0.94      0.94      3389



Confusion Matrix

In [14]:
print(confusion_matrix(testY, y_pred_bool))

[[  39    3    2    0    1    1    7]
 [   2  297    9    0    2    4   40]
 [   1    5  205    1    3    3   14]
 [   0    9    5   35    1    5   10]
 [   1   17    1    0   88    0   16]
 [   0    5    1    0    0 2296    6]
 [   1   15    7    0    5    1  225]]


Some prediction result of unknown images

In [ ]:
def prepare(filepath):
    image = cv2.imread(filepath)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (200, 200))
    return image.reshape(-1,200,200,3)

prediction=model.predict([prepare("WaterDisaster.jpg")])
prediction=prediction.argmax()
print(classes[prediction])

prediction=model.predict([prepare("Earthquake.jfif")])
prediction=prediction.argmax()
print(classes[prediction])

prediction=model.predict([prepare("Drought.jfif")])
prediction=prediction.argmax()
print(classes[prediction])

prediction=model.predict([prepare("FireDisaster.jfif")])
prediction=prediction.argmax()
print(classes[prediction])


Water_Disaster
Earthquake
Drought
Fire_Disaster
